In [1]:
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size
        
    def print(self, lvl):
        print(f'{lvl}- {self.name} (file, size={self.size})')

In [48]:
class Dir:
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.size = 0
        
        self.dirs = {}
        
        self.files = {}
        
    def add_size(self, size):
        self.size += size
        if self.parent is not None:
            self.parent.add_size(size)

    def parse(self, commands):
        while True:
            if len(commands) == 0:
                return
            cmd = commands[0].strip()
            if cmd == '$ cd ..':
                commands.pop(0)
                target_dir = self.parent
                break
            elif cmd == '$ cd /':
                if self.parent is None:
                    commands.pop(0)
                    continue
                target_dir = self.parent
                break
            elif cmd.startswith('$ cd '):
                dirname = cmd.split(' ')[2]
                commands.pop(0)
                target_dir = self.dirs[dirname]
                break
            elif cmd.startswith('dir'):
                dirname = cmd.split(' ')[1]
                commands.pop(0)
                if dirname not in self.dirs:
                    self.dirs[dirname] = Dir(dirname, self)
            elif cmd == '$ ls':
                commands.pop(0)
                continue
            else:
                filesize, filename = cmd.split(' ')
                commands.pop(0)
                if filename not in self.files:
                    self.files[filename] = File(filename, filesize)
                    self.add_size(int(filesize))
        target_dir.parse(commands)
        
    def print(self, lvl):
        print(f'{lvl}- {self.name} (dir, size={self.size})')
        lvl += '  '
        for d in self.dirs.values():
            d.print(lvl)
        for f in self.files.values():
            f.print(lvl)
        

In [76]:
with open('input7.txt', 'r') as inf:
    commands = list(inf)
    
Root = Dir('/', None)

Root.parse(commands)

Root.print('')

- / (dir, size=44274331)
  - ddpgzpc (dir, size=7431308)
    - mlm (dir, size=102897)
      - zfhnw.zpd (file, size=102897)
    - zffbmlbd (dir, size=6835091)
      - bqpwdh (dir, size=1131772)
        - bhmw (dir, size=190433)
          - zbcbr (file, size=190433)
        - lfhgjw (dir, size=98160)
          - ndrcgmd (dir, size=98160)
            - mjm.dhc (file, size=98160)
        - pjqwq (dir, size=815510)
          - dtzw (file, size=50937)
          - mjm.dhc (file, size=186171)
          - mlm (file, size=305433)
          - mlm.rhf (file, size=272969)
        - dtzw (file, size=27669)
      - gqrlmdhs (dir, size=3492509)
        - blc (dir, size=234394)
          - drjdcqw.szd (file, size=53872)
          - ggh.qsl (file, size=115417)
          - pjqwq (file, size=65105)
        - mlm (dir, size=2175544)
          - bqpwdh (dir, size=334790)
            - jzfgz (dir, size=334790)
              - dtzw (file, size=334790)
          - sdjnlsf (dir, size=1038307)
            - vpd

In [77]:
def sum_dirs(cdir):
    csum = 0
    for d in cdir.dirs.values():
        if int(d.size) <= 100000:
            csum += d.size
        csum += sum_dirs(d)
    return csum

In [78]:
print(sum_dirs(Root))

1915606


In [79]:
minspace = 30000000 - (70000000 - Root.size)

In [80]:
def find_dirs(cdir):
    dirsizes = []
    for d in cdir.dirs.values():
        dirsizes.append(d.size)
        dirsizes += find_dirs(d)
    return dirsizes

In [81]:
dirsizes = find_dirs(Root)

In [68]:
dirsizes

[94853, 584, 24933642]

In [82]:
suitable = []
for d in dirsizes:
    if d > minspace:
        suitable.append(d)

In [84]:
min(suitable)

5025657